<a href="https://colab.research.google.com/github/pavansai26/END-TO-END-DEPLOYMENT-NER-/blob/main/deployment_of_ner_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install necessary dependecies

In [6]:
!pip install transformers
!pip install redis
!pip install pymongo
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00


In [12]:
import tensorflow as tf  
# Import TensorFlow library
from transformers import TFXLMRobertaForTokenClassification, XLMRobertaTokenizer  
# Import XLM-RoBERTa model and tokenizer from transformers library
import pymongo  
# Import pymongo library for MongoDB interaction
import redis  
# Import redis library for Redis interaction
from transformers import AutoTokenizer, AutoConfig

# Connect to MongoDB


In [9]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")  
# Connect to local MongoDB instance
db = mongo_client["named_entity_recognition"]  
# Get or create "named_entity_recognition" database
collection = db["sentences"]  
# Get or create "sentences" collection in "named_entity_recognition" database

Connect to Redis

In [10]:
redis_client = redis.Redis(host='localhost', port=6379, db=0)
# Connect to local Redis instance on default port and database 0

Load XLM-RoBERTa model and tokenizer

In [13]:
model_name = 'jplu/tf-xlm-roberta-base'  
# Set name of pre-trained XLM-RoBERTa model to use
tokenizer = AutoTokenizer.from_pretrained(model_name)  
# Instantiate auto tokenizer using pre-trained model
model = TFXLMRobertaForTokenClassification.from_pretrained(model_name)  
# Instantiate XLM-RoBERTa model for token classification using pre-trained model

All model checkpoint layers were used when initializing TFXLMRobertaForTokenClassification.

Some layers of TFXLMRobertaForTokenClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Define named entity recognition function


In [14]:
def recognize_entities(sentence):
    # Check if result is cached in Redis
    if redis_client.exists(sentence):
        cached_result = redis_client.get(sentence)
        return cached_result.decode('utf-8')  
        # Return cached result as a string
    else:
        # Run named entity recognition on sentence
        inputs = tokenizer(sentence, return_tensors='tf')  
        # Tokenize sentence using XLM-RoBERTa tokenizer
        outputs = model(inputs)  
        # Feed tokenized sentence to XLM-RoBERTa model for token classification
        predictions = tf.argmax(outputs.logits, axis=-1).numpy()[0]  
        # Get predicted entity labels for each token in the sentence
        entity_labels = [tokenizer.decode([p]) for p in predictions]  
        # Convert predicted entity labels to strings using tokenizer

        # Combine adjacent tokens with same entity label
        entity_spans = []
        for i in range(len(entity_labels)):
            if entity_labels[i].startswith('B-'):
                start = i
                end = i
                while end < len(entity_labels) - 1 and entity_labels[end+1].startswith('I-'):
                    end += 1
                entity_spans.append((start, end, entity_labels[i][2:]))
        entities = [{'text': tokenizer.decode(inputs['input_ids'][0][start:end+1]), 'type': entity_type} for start, end, entity_type in entity_spans]

        # Cache result in Redis
        redis_client.set(sentence, str(entities), ex=3600)  
        # Cache entities for the input sentence in Redis for 1 hour (3600 seconds)

        return entities  
        # Return list of recognized entities as dictionaries with 'text' and 'type' keys








# Store data in MongoDB


In [ ]:
data = {'sentence': 'John lives in New York City.', 'entities': recognize_entities('John lives in New York City.')}  
# Create dictionary containing sentence and recognized entities
collection.insert_one(data)  
# Insert data into MongoDB "sentences" collection

# Retrieve data from MongoDB
results = collection.find({})  
# Find all documents in MongoDB "sentences" collection
for result in results:
    sentence = result['sentence']
    entities = result['entities']

    if redis_client.exists(sentence):
        cached_result = redis_client.get(sentence)
        print(cached_result.decode('utf-8'))
    else:
        recognized_entities = recognize_entities(sentence)
        print(recognized_entities)  

In [ ]:
# Use an official Python runtime as a parent image
FROM python:3.9-slim-buster

# Set the working directory in the container to /app
WORKDIR /app

# Copy the requirements file into the container at /app
COPY requirements.txt .

# Install the required packages
RUN pip3 install --no-cache-dir -r requirements.txt

# Copy the rest of the application code into the container at /app
COPY . .

# Expose port 5000 for the Flask server
EXPOSE 5000

# Set the environment variables for MongoDB and Redis
ENV MONGO_HOST localhost
ENV MONGO_PORT 27017
ENV REDIS_HOST localhost
ENV REDIS_PORT 6379
ENV REDIS_DB 0

# Run the command to start the Flask server
CMD [ "python3", "app.py" ]


You should also create a requirements.txt file in the same directory as the Dockerfile, which should contain the following packages:

tensorflow
transformers
pymongo
redis
flask


Make sure to replace app.py with the filename of your application code. Then you can build and run the Docker container using the following commands:

docker build -t my_app .
docker run -p 5000:5000 my_app


This will map the container's port 5000 to your machine's port 5000, allowing you to access the Flask server running in the container

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: named-entity-recognition
  labels:
    app: named-entity-recognition
spec:
  replicas: 1
  selector:
    matchLabels:
      app: named-entity-recognition
  template:
    metadata:
      labels:
        app: named-entity-recognition
    spec:
      containers:
      - name: named-entity-recognition
        image: your-docker-image-tag
        ports:
        - containerPort: 8080
        env:
        - name: MONGO_URI
          value: "mongodb://mongodb:27017/named_entity_recognition"
        - name: REDIS_HOST
          value: "redis"
        - name: REDIS_PORT
          value: "6379"
        - name: REDIS_DB
          value: "0"
---
apiVersion: v1
kind: Service
metadata:
  name: named-entity-recognition
  labels:
    app: named-entity-recognition
spec:
  selector:
    app: named-entity-recognition
  ports:
  - name: http
    port: 80
    targetPort: 8080
  type: LoadBalancer
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mongodb
  labels:
    app: mongodb
spec:
  selector:
    matchLabels:
      app: mongodb
  replicas: 1
  template:
    metadata:
      labels:
        app: mongodb
    spec:
      containers:
      - name: mongodb
        image: mongo:4.4.6
        ports:
        - containerPort: 27017
        env:
        - name: MONGO_INITDB_DATABASE
          value: named_entity_recognition
---
apiVersion: v1
kind: Service
metadata:
  name: mongodb
  labels:
    app: mongodb
spec:
  ports:
  - name: mongo
    port: 27017
  selector:
    app: mongodb
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis
  labels:
    app: redis
spec:
  selector:
    matchLabels:
      app: redis
  replicas: 1
  template:
    metadata:
      labels:
        app: redis
    spec:
      containers:
      - name: redis
        image: redis:6.2.4
        ports:
        - containerPort: 6379
---
apiVersion: v1
kind: Service
metadata:
  name: redis
  labels:
    app: redis
spec:
  ports:
  - name: redis
    port: 6379
  selector:
    app: redis


This YAML file will deploy the named-entity-recognition Docker container, along with MongoDB and Redis containers as dependencies. The named-entity-recognition container will listen on port 8080, and a LoadBalancer service will expose it on port 80. The MongoDB container will listen on port 27017, and the Redis container will listen on port 6379. The MONGO_URI, REDIS_HOST, REDIS_PORT, and REDIS_DB environment variables are passed to the named-entity-recognition container, which are used by the Python code to connect to MongoDB and Redis.

Please replace your-docker-image-tag in the YAML file with the tag of your built Docker image.

kubectl apply -f <FILE_NAME>.yaml
